# 🏙️ Urban Growth GPU Pipeline (Simple Version)
This notebook loads the Austin Construction Permits CSV (~2GB), extracts useful insights, 
and prepares a clean dataset for downstream models (RAG, clustering, predictions, etc.).

In [ ]:
import cudf
import cupy as cp
import numpy as np
import re
from cudf.core.column import as_column

data_path = "/home/asus/atx-hackathon/data/Issued_Construction_Permits_20251213.csv"
print("Loading file:", data_path)

## 🔹 Step 1 — Load CSV into cuDF (GPU)

In [ ]:
df = cudf.read_csv(data_path, low_memory=True)
df.head()

## 🔹 Step 2 — Basic Cleanup
- Normalize text fields
- Fill missing values
- Create lower‑case searchable versions

In [ ]:
text_cols = ["DESCRIPTION", "WORK_CLASS", "ISSUED_DATE"]

for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna("")
        df[col + "_clean"] = df[col].str.lower()
        
df.head()

## 🔹 Step 3 — Feature Extraction (Simple Patterns)
We create quick flags for:
🏠 ADUs (Accessory Dwelling Units)
🔌 EV Chargers
🏢 Multi‑family projects
☀️ Solar installs
🛠️ Renovations

In [ ]:
def make_flag(col, pattern):
    return df[col].str.contains(pattern, regex=True)

df["flag_adu"]   = make_flag("DESCRIPTION_clean", r"\badu\b| accessory dwelling")
df["flag_ev"]    = make_flag("DESCRIPTION_clean", r"ev charger|electric vehicle")
df["flag_multi"] = make_flag("DESCRIPTION_clean", r"multi[- ]family|apt|apartment")
df["flag_solar"] = make_flag("DESCRIPTION_clean", r"solar|pv system")
df["flag_reno"]  = make_flag("DESCRIPTION_clean", r"renovation|remodel|repair")

df[["flag_adu", "flag_ev", "flag_multi", "flag_solar", "flag_reno"]].head()

## 🔹 Step 4 — Quick Insights
Counts of major project types.

In [ ]:
counts = {
    "ADUs": int(df.flag_adu.sum()),
    "EV Chargers": int(df.flag_ev.sum()),
    "Multi-Family": int(df.flag_multi.sum()),
    "Solar": int(df.flag_solar.sum()),
    "Renovations": int(df.flag_reno.sum())
}

counts

## 🔹 Step 5 — Save a Clean Parquet File
This output can be used directly by your RAG/chatbot or your ML forecasting pipeline.

In [ ]:
output_path = "/home/asus/atx-hackathon/data/permits_clean.parquet"
df.to_parquet(output_path)
print("Saved:", output_path)

## ✔️ Complete!
You now have:
- GPU-loaded 2M row dataset
- Cleaned text
- Flags for major urban trends
- Summary stats
- Parquet file ready for RAG, analysis, or clustering